# Data Source

The dataset is collected from [Yelp Open Dataset](https://www.yelp.com/dataset) twice in year 2017 and 2018 respectively. The [Dataset Documentation](https://www.yelp.com/dataset/documentation/main) contains information on the structure of the dataset.

In [1]:
# Import python packages
import us
import pandas as pd

## Business Data

In [2]:
# Read in 2017 and 2018 data
business_2017 = pd.read_json('business_2017.json', lines=True)
business_2018 = pd.read_json('business_2018.json', lines=True)

In [3]:
# Print the dimension of data
business_2017.shape

(174567, 15)

In [4]:
# Display first few row of business data
business_2017.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


## Merge Data

In [5]:
# Rename is_open columns
business_2017.rename(columns={'is_open':'is_open_2017'}, inplace=True)
business_2018.rename(columns={'is_open':'is_open_2018'}, inplace=True)

In [6]:
# Filter columns in 2018 business data
var_to_keep = ['business_id', 'is_open_2018']
business_2018 = business_2018[var_to_keep]

In [7]:
# Merge two dataframe based on business id
business = business_2017.merge(business_2018, how='inner', on='business_id')
business.shape

(159490, 16)

## Create Bankruptcy Label

In [8]:
# Extract all open business in 2017 and create bankruptcy label
business_df = business[business['is_open_2017'] == 1].reset_index().drop('index', axis=1)
business_df['isBankrupt'] = (business_df['is_open_2017'] != business_df['is_open_2018']).astype(int)

In [9]:
business_df.shape

(133828, 17)

## Extract Restaurant Data

In [10]:
# Filter restaurant data
business_df = business_df[business_df['categories'].apply(lambda row: 'Restaurants' in str(row))]
business_df.shape

(36512, 17)

## Extract United States Restaurants

In [11]:
# Keep only US restaurant data
us_states = us.states.mapping('name','abbr')
business_df = business_df[business_df['state'].apply(lambda row: row in us_states.values())]
business_df.shape

(22832, 17)

In [12]:
# Compute the count of each US state
state_count = business_df['state'].value_counts()
state_count

AZ    7183
NV    4780
OH    3695
NC    2837
PA    2655
WI    1100
IL     434
SC     138
NY       9
AK       1
Name: state, dtype: int64

In [13]:
# Keep only the US states with most number of restaurant
state_to_keep = state_count.index[0]
business_df = business_df[business_df['state'].apply(lambda row: row == state_to_keep)]
business_df.shape

(7183, 17)

In [14]:
# Extract restaurant ids from business data
restaurant_ids = business_df['business_id'].tolist()

## Review Data

In [15]:
# Count the number of line in review data
! wc -l review_2017.json

5261669 review_2017.json


In [16]:
# Save the review line count into a python variable and read in dataset using pandas reader
review_count = 5261669
review_reader = pd.read_json('review_2017.json', lines=True, chunksize=50000)

In [17]:
# Define function that filters the dataset using loop
def df_filter(reader, num_line):
    
    # Calculate the number of iteration required
    if num_line % reader.chunksize == 0:
        num_iteration = num_line // reader.chunksize
    else: 
        num_iteration = num_line // reader.chunksize + 1

    # Loop through the dataset and filter all restaurant business
    for i in range(num_iteration):
        temp = next(reader)
        temp = temp[temp['business_id'].apply(lambda row: row in restaurant_ids)]
        
        # Append the resulting dataframe into the original one if already exists
        try:
            df = pd.concat([df, temp])
        except:
            df = temp
    
    return df

In [18]:
%%time
# Filter the review data
review_df = df_filter(review_reader, review_count)
print(review_df.shape)

(781473, 9)
Wall time: 18min 18s


In [19]:
# Save the resulting datasets into csv files
business_df.to_csv('yelp_business.csv', index=False)
review_df.to_csv('yelp_review.csv', index=False)